# Source imaging / Inverse problem

In [1]:
import mne
from config_import import get_events, event_id
import mne_bids

%matplotlib qt

In [2]:
ls bids_data/sub-09/meg

sub-09_coordsystem.json               sub-09_task-POGS_run-05_channels.tsv
sub-09_task-POGS_run-01_channels.tsv  sub-09_task-POGS_run-05_events.tsv
sub-09_task-POGS_run-01_events.tsv    sub-09_task-POGS_run-05_meg.fif
sub-09_task-POGS_run-01_meg.fif       sub-09_task-POGS_run-05_meg.json
sub-09_task-POGS_run-01_meg.json      sub-09_task-POGS_run-06_channels.tsv
sub-09_task-POGS_run-02_channels.tsv  sub-09_task-POGS_run-06_events.tsv
sub-09_task-POGS_run-02_events.tsv    sub-09_task-POGS_run-06_meg.fif
sub-09_task-POGS_run-02_meg.fif       sub-09_task-POGS_run-06_meg.json
sub-09_task-POGS_run-02_meg.json      sub-09_task-POGS_run-07_channels.tsv
sub-09_task-POGS_run-03_channels.tsv  sub-09_task-POGS_run-07_events.tsv
sub-09_task-POGS_run-03_events.tsv    sub-09_task-POGS_run-07_meg.fif
sub-09_task-POGS_run-03_meg.fif       sub-09_task-POGS_run-07_meg.json
sub-09_task-POGS_run-03_meg.json      sub-09_task-POGS_run-08_channels.tsv
sub-09_task-POGS_run-04_channels.tsv  sub-09_task-POGS_run

In [3]:
subject = '09'
session = None
task = 'POGS'
run = '01'
bids_root = './bids_data'

bids_path = mne_bids.BIDSPath(
    subject=subject,
    session=session,
    run=run,
    task=task,
    datatype='meg',
    suffix='meg',
    extension='.fif',
    root=bids_root
)

raw = mne_bids.read_raw_bids(bids_path, extra_params=dict(allow_maxshield="yes"))

/Users/alex/work/src/mne-bids/mne_bids/read.py:564: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015, STI016, STI101, STI201, STI301 has changed from V to NA.
  raw.set_channel_types(ch_type_map_avail)


In [4]:
raw.annotations

<Annotations | 330 segments: hinge/outlier/1 (1), hinge/outlier/2 (1), ...>

In [5]:
bads = ['MEG1613', 'MEG0213', 'MEG0311', 'MEG0813', 'MEG1542', 'MEG1541',
        'MEG1811', 'MEG0623', 'MEG0241']
raw.info['bads'] = bads

In [6]:
cal_fname = "meg_conf/sss_cal_nsp_2017.dat"
ct_fname = "meg_conf/ct_sparse_nsp_2017.fif"
raw = mne.preprocessing.maxwell_filter(
    raw, origin="auto", calibration=cal_fname, cross_talk=ct_fname)

In [7]:
l_freq, h_freq = 0.1, 40
raw.filter(l_freq, h_freq)

<Raw | sub-09_task-POGS_run-01_meg.fif, 336 x 344000 (344.0 s), ~887.0 MB, data loaded>

In [8]:
events, event_id = mne.events_from_annotations(raw)

In [9]:
epochs = mne.Epochs(raw, events, event_id, tmin=-0.1, tmax=1., baseline=None)

In [10]:
evoked = epochs["reference"].average()
evoked.plot_joint()

[<Figure size 1600x840 with 7 Axes>, <Figure size 1600x840 with 7 Axes>]

# Now let's look at the MRI data

In [11]:
subjects_dir = "bids_data/derivatives/freesurfer/subjects"
fs_subject = f"sub-{subject}"

In [12]:
mne.viz.plot_bem(subject=fs_subject, subjects_dir=subjects_dir,
                 orientation='coronal');

In [13]:
# trans = mne.read_trans('pogs_meg/ll_180197/220217/sub-09-trans.fif')
mri_bp = bids_path.copy().update(run=None, datatype='anat', suffix='T1w', extension=".nii.gz")
trans = mne_bids.get_head_mri_trans(bids_path, t1_bids_path=mri_bp, fs_subjects_dir=subjects_dir)
trans

/Users/alex/work/src/mne-bids/mne_bids/read.py:931: DeprecationWarning: Passing "fs_subject=None" has been deprecated and will raise an error in future versions. Please explicitly specify the FreeSurfer subject name.
  warn('Passing "fs_subject=None" has been deprecated and will raise '
/Users/alex/work/src/mne-bids/mne_bids/read.py:57: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = reader[ext](raw_path, allow_maxshield, **kwargs)
/Users/alex/work/src/mne-bids/mne_bids/read.py:564: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015, STI016, STI101, STI201, STI301 has changed from V to NA.
  raw.set_channel_types(ch_type_map_avail)


<Transform | head->MRI (surface RAS)>
[[ 0.99935293 -0.01144447  0.03409905  0.000616  ]
 [ 0.01518899  0.99363005 -0.11166293 -0.00605521]
 [-0.03260392  0.1121086   0.99316094 -0.01032828]
 [ 0.          0.          0.          1.        ]]

In [14]:
fig = mne.viz.plot_alignment(evoked.info, trans, subject=fs_subject, dig=True,
                             subjects_dir=subjects_dir, verbose=True);

Using pyvistaqt 3d backend.

Using outer_skin.surf for head surface.
Getting helmet for system 306m
Channel types::	grad: 204, mag: 102


In [15]:
# mne.gui.coregistration(subject=fs_subject, subjects_dir=subjects_dir, inst=bids_path)

# Now let's do some source imaging

In [16]:
bids_path_er = bids_path.find_empty_room()

raw_er = mne_bids.read_raw_bids(bids_path_er, extra_params=dict(allow_maxshield="yes"))
raw_er.info["bads"] = bads
raw_er.info["dev_head_t"] = raw.info["dev_head_t"]
raw_er.set_montage(raw.get_montage())
raw_er = mne.preprocessing.maxwell_filter(
    raw_er, origin="auto", calibration=cal_fname, cross_talk=ct_fname,
)
raw_er.filter(l_freq, h_freq)

/Users/alex/work/src/mne-bids/mne_bids/read.py:57: RuntimeWarning: This file contains raw Internal Active Shielding data. It may be distorted. Elekta recommends it be run through MaxFilter to produce reliable results. Consider closing the file and running MaxFilter on the data.
  raw = reader[ext](raw_path, allow_maxshield, **kwargs)
/Users/alex/work/src/mne-bids/mne_bids/read.py:564: RuntimeWarning: The unit for channel(s) STI001, STI002, STI003, STI004, STI005, STI006, STI007, STI008, STI009, STI010, STI011, STI012, STI013, STI014, STI015, STI016, STI101, STI201, STI301 has changed from V to NA.
  raw.set_channel_types(ch_type_map_avail)
/Users/alex/work/src/mne-bids/mne_bids/path.py:1483: RuntimeWarning: Did not find any events.tsv associated with sub-emptyroom_ses-20220217_task-noise.

The search_str was "bids_data/sub-emptyroom/**/meg/sub-emptyroom_ses-20220217*events.tsv"
  warn(msg)
/Users/alex/work/src/mne-bids/mne_bids/read.py:564: RuntimeWarning: The unit for channel(s) STI00

<Raw | sub-emptyroom_ses-20220217_task-noise_meg.fif, 336 x 100000 (100.0 s), ~256.8 MB, data loaded>

In [17]:
raw_er.plot_psd(fmax=60);

In [18]:
cov = mne.compute_raw_covariance(raw_er)

In [19]:
mne.viz.plot_cov(cov, info=raw_er.info)

(<Figure size 1520x740 with 4 Axes>, <Figure size 1520x740 with 2 Axes>)

In [20]:
evoked.plot_white(noise_cov=cov);

In [21]:
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject=fs_subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

In [22]:
src = mne.setup_source_space(fs_subject, spacing='oct6',
                             subjects_dir=subjects_dir,
                             add_dist=False)

In [23]:
info = evoked.info
fwd = mne.make_forward_solution(info, trans=trans,
                                src=src, bem=bem,
                                meg=True, # include MEG channels
                                eeg=False, # include EEG channels
                                mindist=5.0, # ignore sources <= 5mm from inner skull
                                n_jobs=1) # number of jobs to run in parallel

In [24]:
from mne.minimum_norm import make_inverse_operator, apply_inverse

inverse_operator = make_inverse_operator(info, fwd, cov,
                                         loose=0.2, depth=0.8)

In [25]:
method = "dSPM"
snr = 3.
lambda2 = 1. / snr ** 2
stc = apply_inverse(evoked, inverse_operator, lambda2,
                    method=method, pick_ori=None)
stc

<SourceEstimate | 8195 vertices, subject : sub-09, tmin : -100.0 (ms), tmax : 1000.0 (ms), tstep : 1.0 (ms), data shape : (8195, 1101), ~68.9 MB>

In [26]:
brain = stc.plot(surface='inflated', views=['lat', 'med'], hemi='split', subjects_dir=subjects_dir)